In [ ]:
import os
import glob
from dotenv import load_dotenv
import gradio as gr

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.llms import Ollama

In [4]:
db_name = "vector_db"

In [3]:
folders = glob.glob("knowledge_base/*")

In [ ]:
def add_metadata(document, doc_type):
    document.metadata["doc_type"] = doc_type
    return document

In [ ]:
documents = []
for folder in folders:
    # get the folder name
    doc_type = os.path.basename(folder)
    # use the DirectoryLoader to load all documents in the folder
    loader = DirectoryLoader(folder, glob="*.txt", loader_cls=TextLoader, loader_kwargs={'encoding': 'utf-8'})
    # load the documents and add metadata
    folder_documents = loader.load()
    # add metadata to each document
    documents.extend([add_metadata(doc, doc_type) for doc in folder_documents])

# Split documents into smaller chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunked_documents = text_splitter.split_documents(documents)

print(f"Total number of chunks: {len(chunked_documents)}")
print(f"Document types found: {set(doc.metadata['doc_type'] for doc in documents)}")

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create vectorstore
vectorstore = Chroma.from_documents(documents=chunked_documents, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

In [ ]:
llm = ChatOpenAI(temperature=0.7, model_name='llama3.2', base_url='http://localhost:11434/v1', api_key='ollama')
# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever(search_kwargs={"k": 25})
# putting it together: set up the conversation chain with the LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [ ]:
def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

In [ ]:
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)